In [26]:
!pip install pyspark

In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, isnull, lit

from pyspark.sql import functions as F
from pyspark.sql.types import StringType


from pyspark.sql.functions import col, desc

In [28]:
# Create a SparkSession
spark = SparkSession.builder.appName("SalesDataProcessing").getOrCreate()

csv_path = "/content/supermarket_sales - Sheet1.csv"



In [29]:

df = spark.read.csv(csv_path, header=True, inferSchema=True)


df = df.fillna(0, subset=["Total"])

df = df.withColumn("Customer type", F.coalesce(col("Customer type"), F.lit("Unknown")))

df = df.filter(col("Product Line") != "")


df = df.dropDuplicates()


filtered_df = df.filter(df["Quantity"]>5)

# Group by Product line and calculate total sales amount
product_sales = df.groupBy("Product line").agg({"Total":"sum"})


sorted_df = df.orderBy(df["Unit price"].desc())

In [32]:
total_sales = df.selectExpr("sum(Total)").collect()[0][0]
print("Total Sales:", total_sales)


Total Sales: 322966.7489999997


In [34]:
data = df.collect()

print(data)

[Row(Invoice ID='280-35-5823', City='Mandalay', Customer type='Member', Product line='Home and lifestyle', Unit price=86.72, Quantity=1, Tax 5%=4.336, Total=91.056, Date='1/17/2019', Payment='Ewallet', gross margin percentage=4.761904762, gross income=4.336, Rating=7.9), Row(Invoice ID='189-08-9157', City='Naypyitaw', Customer type='Normal', Product line='Fashion accessories', Unit price=31.73, Quantity=9, Tax 5%=14.2785, Total=299.8485, Date='01-08-2019', Payment='Credit card', gross margin percentage=4.761904762, gross income=14.2785, Rating=5.9), Row(Invoice ID='275-28-0149', City='Yangon', Customer type='Normal', Product line='Sports and travel', Unit price=63.69, Quantity=1, Tax 5%=3.1845, Total=66.8745, Date='2/25/2019', Payment='Cash', gross margin percentage=4.761904762, gross income=3.1845, Rating=6.0), Row(Invoice ID='442-48-3607', City='Yangon', Customer type='Member', Product line='Food and beverages', Unit price=23.48, Quantity=2, Tax 5%=2.348, Total=49.308, Date='3/14/201

In [ ]:
# Calculate total sales amount for each product
product_sales = df.groupBy("Product Line").agg({"Total": "sum"}).withColumnRenamed("sum(Sum Total)", "total_sales")

# Define the output path for the new CSV file
output_path = "total_sales.csv"

# Write the results to a new CSV file
product_sales.write.csv(output_path, header=True)

# Stop the SparkSession
spark.stop()
